# CS/ECE/ISyE 524 - Spr 2018 - HW 8 - Solutions
### Prepared by: Laurent Lessard

# 1. Thrift store

How should you make change for 99 cents if the goal is to minimize the total weight of the coins used? The following table shows the weight of each type of coin. You may use any number of each type of coin.

| Type of coin | penny | nickel | dime | quarter |
|:------------:|:-----:|:------:|:----:|:-------:|
|Weight (grams)| 2.500 |  5.000 |2.268 |  5.670  |

In [2]:
coinType = [ :penny, :nickel, :dime, :quarter ]

# problem data (we'll use a dictionary here)
weights = Dict( :penny => 2.500, :nickel => 5.000, :dime => 2.268, :quarter => 5.670 )
values  = Dict( :penny => 1    , :nickel => 5    , :dime => 10   , :quarter => 25    )

using JuMP, Cbc
m = Model(solver = CbcSolver())

@variable(m, x[coinType] >= 0, Int)
@constraint(m, sum( x[i] * values[i] for i in coinType ) == 99 )
@objective(m, Min, sum( x[i] * weights[i] for i in coinType ) )
solve(m)

println(getvalue(x))
println(getobjectivevalue(m))

x: 1 dimensions:
[  penny] = 4.0
[ nickel] = 0.0
[   dime] = 2.0
[quarter] = 3.0
31.546


# 2. Checked luggage.

You're trying to pack as many souvenirs as possible to bring home from your trip, but your suitcase has a limited capacity. It can hold a maximum of 30 pounds of weight and 15 gallons of volume. Which souvenirs should you pack? The weights and volumes are as follows:

|Souvenir number |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|
|:---------------|--|--|--|--|--|--|--|--|--|--|
|Weight (lbs)    |5 |6 |7 |6 |4 |6 |7 |3 |8 |5 |
|Volume (gal)    |2 |4 |5 |3 |3 |2 |3 |1 |2 |4 |

In [3]:
# problem data (we'll use an aray here)
weights = [ 5, 6, 7, 6, 4, 6, 7, 3, 8, 5 ]
volumes = [ 2, 4, 5, 3, 3, 2, 3, 1, 2, 4 ]
n = length(volumes) # total number of souvenirs

using JuMP, Cbc
m = Model(solver = CbcSolver())

@variable(m, x[1:n], Bin)
@constraint(m, sum( x[i] * weights[i] for i=1:n ) <= 30 )
@constraint(m, sum( x[i] * volumes[i] for i=1:n ) <= 15 )
@objective(m, Max, sum(x) )
solve(m)

println(getvalue(x))
println(getobjectivevalue(m))

[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
6.0


# 3. Comquat Computers

Comquat owns four production plants at which personal computers are produced. Comquat can sell up to 20,000 computers per year at a price of \$3,500 per computer. For each plant the production capacity, cost per computer, and fixed cost of operating the plant for a year are given below. Determine how Comquat can maximize its yearly profit from computer production.

| Plant | Production capacity | Plant fixed cost (\$ Million) | Cost per computer (\$) |
|:-----:|:-------------------:|:-----------------------------:|:----------------------:|
| 1     | 10,000              |   9                           |  1,000                 |
| 2     |  8,000              |   5                           |  1,700                 |
| 3     |  9,000              |   3                           |  2,300                 |
| 4     |  6,000              |   1                           |  2,900                 |

In [4]:
using JuMP, Cbc
m = Model(solver = CbcSolver())

price     = 3.5e3                       # price at which we can sell a single computer (regardless of where it is produced)
cap       = 20000                       # maximum computers that can be sold per year

capacity  = [10000, 8000, 9000, 6000]   # production capacity for each plant
fixedcost = [9e6, 5e6, 3e6, 1e6]        # fixed cost for each plant
itemcost  = [1e3, 1.7e3, 2.3e3, 2.9e3]  # cost per computer for each factory

@variable(m, z[1:4], Bin)   # indicator variables: is plant i on or off?
@variable(m, x[1:4] >= 0)   # number of computers produced at each of the plants

# production constraints on each plant
@constraint(m, x .<= capacity)

# total production capacity constraint
@constraint(m, sum(x) <= cap)

# logic constraint: if z[i]=0 then we must have x[i]=0
@constraint(m, x .<= capacity .* z)

# expenses (item costs + fixed costs)
@expression(m, expenses, sum( itemcost[i]*x[i] for i in 1:4) + sum( fixedcost[i]*z[i] for i in 1:4) )

# revenue (from selling items)
@expression(m, revenue, price*sum(x))

# MAXIMIZE PROFIT (revenue - expenses)
@objective(m, Max, revenue - expenses)

solve(m)
println("The maximal profit is: ", getobjectivevalue(m))
println("Here is a tally of factory number, whether it's open or not, and how many computers it should produce.")
getvalue([1:4 z x])

The maximal profit is: 2.56e7
Here is a tally of factory number, whether it's open or not, and how many computers it should produce.


4×3 Array{Float64,2}:
 1.0  1.0  10000.0
 2.0  1.0   8000.0
 3.0  0.0      0.0
 4.0  1.0   2000.0

# 4. ABC Investments

ABC Inc. is considering several investment options.  Each option has a minimum investment required as well as a maximum investment allowed. These restrictions, along with the expected return are summarized in the following table (figures are in millions of dollars):

|Option | Minimum investment | Maximum investment | Expected return (%)  |
|:-----:|:------------------:|:------------------:|:--------------------:|
|  1    |      3             |      27            |     13               |
|  2    |      2             |      12            |      9               |
|  3    |      9             |      35            |     17               |
|  4    |      5             |      15            |     10               |
|  5    |     12             |      46            |     22               |
|  6    |      4             |      18            |     12               |

Because of the high-risk nature of Option 5, company policy requires that the total amount invested in Option 5 be no more that the combined amount invested in Options 2, 4 and 6.  In addition, if an investment is made in Option 3, it is required that at least a minimum investment be made in Option 6.  ABC has \$80 million to invest and obviously wants to maximize its total expected return on investment.  Which options should ABC invest in, and how much should be invested?

In [5]:
using JuMP, Cbc
m = Model(solver = CbcSolver())

lows  = [  3,  2,  9,  5, 12,  4 ]  # minimum investment
highs = [ 27, 12, 35, 15, 46, 18 ]  # maximum investment
ret   = [ 13,  9, 17, 10, 22, 12 ]  # expected return

@variable(m, x[1:6] >= 0)             # amount invested in each option
@variable(m, z[1:6], Bin)             # indicator variable: do we invest in option i or not?

@constraint(m, sum(x) <= 80)     # total investment limit

# option 5 is no more than combined amounts from options 2, 4, and 6
@constraint(m, x[5] <= x[2] + x[4] + x[6])

# if there is an investment in option 3, we must invest in option 6.
# in other words: if z[3] = 1, then z[6] = 1.
@constraint(m, z[3] <= z[6])

# variable lower bounds (if z=0 then x=0. if z=1 then low <= x <= hi)
for i = 1:6
    @constraint(m, lows[i]*z[i] <= x[i]  )
    @constraint(m, x[i] <= highs[i]*z[i] )
end

@expression(m, expectedReturn, sum( x[i] * ret[i]/100 for i = 1:6) )

@objective(m, Max, expectedReturn)

solve(m)
println("The maximum return on investment (net profit) is: \$", getobjectivevalue(m), " million.")
println("This is an average return of: ", 100*getobjectivevalue(m)/getvalue(sum(x)), " %")
println("Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of \$).")
getvalue([1:6 z x])

The maximum return on investment (net profit) is: $13.5 million.
This is an average return of: 16.875 %
Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of $).


6×3 Array{Float64,2}:
 1.0  0.0   0.0
 2.0  0.0   0.0
 3.0  1.0  35.0
 4.0  1.0   5.0
 5.0  1.0  22.5
 6.0  1.0  17.5